In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train=pd.read_csv('/kaggle/input/ieee-fraud-detection/train_transaction.csv')
train_id=pd.read_csv('/kaggle/input/ieee-fraud-detection/train_identity.csv')

In [ ]:
categorical_id_cols=train_id.dtypes[train_id.dtypes==np.dtype('O')].index
categorical_cols=train.dtypes[train.dtypes==np.dtype('O')].index

In [ ]:
categorical_cols

In [ ]:
train.TransactionID.unique().shape

In [ ]:
train_id.isna().sum()

In [ ]:
na=train.isna().sum()/train.shape[0]
na_cols=na[na>0.1].index
na_cols

In [ ]:
train[categorical_cols]=train[categorical_cols].fillna('none')

In [ ]:
train_id[categorical_id_cols]=train_id[categorical_id_cols].fillna('none')

In [ ]:
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder

In [ ]:
na_cols=set(na_cols)-set(categorical_cols)
train.drop(columns=na_cols,inplace=True)

In [ ]:
oe=OrdinalEncoder()
df=oe.fit_transform(train[categorical_cols])
df=pd.DataFrame(df, columns=categorical_cols, index=train.index)
df=pd.concat([train.drop(columns=categorical_cols),df],axis=1)

In [ ]:
oe_id=OrdinalEncoder()
dg=oe_id.fit_transform(train_id[categorical_id_cols])
dg=pd.DataFrame(dg,columns=categorical_id_cols,index=train_id.index)
dg=pd.concat([train_id.drop(columns=categorical_id_cols),dg],axis=1)

In [ ]:
train_id.columns

In [ ]:
train_id[train_id.TransactionID==2987004]

In [ ]:
df_merge=pd.merge(left=df,right=dg, on='TransactionID',how='left')

## Modeling

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, f1_score
from xgboost import XGBClassifier

In [ ]:
train.columns

In [ ]:
X=df_merge.drop(columns=['TransactionID','isFraud'])
y=df_merge['isFraud']

In [ ]:
xtrain,xtest,ytrain,ytest=train_test_split(X,y,test_size=0.2,stratify=y)

In [ ]:
xgb=XGBClassifier(n_estimators=1000,max_depth=3,n_jobs=-1)

In [ ]:
xgb.fit(xtrain,ytrain)

In [ ]:
p=xgb.predict_proba(xtest)[:,1]
roc_auc_score(ytest,p)